<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/10%20-%20Seq2Seq%20e%20Machine%20Translation/machine_translation_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Architetture Sequence 2 Sequence per la traduzione automatica
In questo notebook creeremo un'architettura **Sequence 2 Sequence** di rete neurale artificiale in grado di tradurre del testo dall'italiano all'inglese, per farlo avremo bisogno di un corpus di testo contenente frasi di esempio in entrambe le lingue, fortunatamente il software di flashcard Anki ci mette a disposizione tali corpus per molteplici lingue, scarichiamo quello per inglese-italiano da [questo link](http://www.manythings.org/anki/). Se usi Google colab o hai wget installato, scarica pure il file zip eseguendo la cella di codice qui sotto.

In [0]:
!wget http://www.manythings.org/anki/ita-eng.zip

--2019-04-20 13:21:25--  http://www.manythings.org/anki/ita-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3981147 (3.8M) [application/zip]
Saving to: ‘ita-eng.zip.1’

ita-eng.zip.1       100%[===================>]   3.80M  1.90MB/s    in 2.0s    

2019-04-20 13:21:27 (1.90 MB/s) - ‘ita-eng.zip.1’ saved [3981147/3981147]



ed estrai lo zip

In [0]:
!unzip ita-eng.zip

Archive:  ita-eng.zip
replace ita.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ita.txt                 
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: _about.txt              


Adesso abbiamo il file ita.txt contenente coppie di frasi in inglese e italiano su ogni riga, separate da un tab. Dividiamo le frasi in base alla lingua in due liste separate e salviamo ogni carattere all'interno di un set, mentre scorriamo il file processiamo anche le frasi, rimuovendo la punteggiatura e convertendo tutto in minuscolo.

In [0]:
import re

en_chars = set({})
it_chars = set({})

it_sents = []
en_sents = []

with open("ita.txt") as en_it_sents:
  lines = en_it_sents.read().split("\n")
  
  for line in lines[:min(num_samples, len(lines)-1)]:
    
    line = re.sub(r'[^\w\s]','',line)
    line = line.lower()
    
    en_sent, it_sent = line.split("\t")
    en_sent = "\t"+en_sent+"\n"
    
    en_sents.append(en_sent)
    it_sents.append(it_sent)
    
    for char in en_sent:
      en_chars.add(char)
      
    for char in it_sent:
      it_chars.add(char)

Convertiamo i set di caratteri in liste ordinate e contiamo il numer do caratteri contenuti in ogni lista.

In [0]:
it_chars = sorted(list(it_chars))
en_chars = sorted(list(en_chars))

num_encoder_tokens = len(it_chars)
num_decoder_tokens = len(en_chars)

print('Numbero di token di input:', num_encoder_tokens)
print('Number di token di output:', num_decoder_tokens)

Contiamo la lungezza massima di una frase per entrambe le lingue.

In [0]:
max_encoder_seq_length = max([len(sent) for sent in it_sents])
max_decoder_seq_length = max([len(sent) for sent in en_sents])

print("Lunghezza massima della sequenza per l'input:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 40
Number of unique output tokens: 37
Max sequence length for inputs: 39
Max sequence length for outputs: 14


Adesso dobbiamo eseguire il one hot encoding di ogni frase a livello di carattere, per farlo ci conviene creare il dizionario che ci permette di accedere velocemente all'indice partendo dal carattere.

In [0]:
it_token_index = dict(
    [(char, i) for i, char in enumerate(it_chars)])
en_token_index = dict(
    [(char, i) for i, char in enumerate(en_chars)])

Adesso eseguiamo il one hot encoding per creare:
- **input dell'encodere**: le frasi italiane.
- **input del decoder**: le frasi in inglese.
- **output del decoder**: le frasi in inglese shiftate di un carattere.

In [0]:
import numpy as np

# inizializziamo gli array vuoti

encoder_input_data = np.zeros(len(it_sents), max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = np.zeros((len(it_sents), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros((len(it_sents), max_decoder_seq_length, num_decoder_tokens))

# iteriamo simultaneamente su frasi in italiano e inglese

for i, (it_sent, en_sent) in enumerate(zip(it_sents, en_sents)):
    for t, char in enumerate(it_sent):
        # assegnamo un 1 all'indice di ogni carattere contenuto nella frase
        encoder_input_data[i, t, it_token_index[char]] = 1.
    for t, char in enumerate(en_sent):
        # assegnamo un 1 all'indice di ogni carattere contenuto nella frase
        decoder_input_data[i, t, en_token_index[char]] = 1.
        if t > 0:
            # shiftiamo di uno il target
            decoder_target_data[i, t - 1, en_token_index[char]] = 1.

Adesso i nostri dati sono pronti, possiamo passare alla creazione del modello.

## Creazione del Modello
Il modello che andremo a creare è molto complesso e alcuni dei suoi strati devono accettare input da strati differenti (il decoder prende lo stato dall'encoder e come input le frasi in inglese), dobbiamo utilizzare le [API Funzionali di Keras](https://keras.io/getting-started/functional-api-guide/). 
<br>
Cominciamo con l'**encoder**:
- Usiamo la classe **Input** per definire l'input dell'encoder.
- Creiamo lo strato ricorrente (LSTM) dell'encoder, che dovrà restituire lo stato (return_state=True)
- Usiamo l'encoder, scartiamo l'ouput (primo array ritornato) e teniamo solo lo stato (secondo e terzo array)


In [0]:
from keras.layers import Input, LSTM, Dense

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

Adesso creiamo il **decoder**:
- Usiamo la classe **Input** per definire l'input dell decoder.
- Creiamo lo strato ricorrente (LSTM) dell'encoder, che dovrà usare lo stato dell'encoder (initial_state=encoder_states)
- Usiamo l'encoder, scartiamo l'ouput (primo array ritornato) e teniamo solo lo stato (secondo e terzo array).
- Creiamo uno strato di output che utilizzerà la funzine di attivazione softmax per eseguire una classificazione multiclasse.


In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(256, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Adesso usiamo la classe **Model** di keras per creare il modello, passando gli strati di input (sia quello dell'encoder che quello del decoder) all'interno di una lista e lo strato di output.

In [0]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Compiliamo il modello, usiamo come algoritmo di ottimizzazione 'rmsprop', il quale dovrebbe portare risultati migliori quando si lavora con modelli sequenziali.

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])

Ed avviamo l'addestramento, qui dobbiamo passare sempre i dati di input, sia per l'encoder che per il decoder, all'interno di una lista e i dati di output.

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=256,
          epochs=100,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 11s 1ms/step - loss: 2.0902 - acc: 0.1964 - val_loss: 2.2545 - val_acc: 0.2288
Epoch 2/100
8000/8000 [==============================] - 10s 1ms/step - loss: 1.7717 - acc: 0.2689 - val_loss: 2.0283 - val_acc: 0.2942
Epoch 3/100
8000/8000 [==============================] - 10s 1ms/step - loss: 1.5651 - acc: 0.3150 - val_loss: 1.8585 - val_acc: 0.3499
Epoch 4/100
8000/8000 [==============================] - 9s 1ms/step - loss: 1.4421 - acc: 0.3442 - val_loss: 1.8498 - val_acc: 0.3484
Epoch 5/100
8000/8000 [==============================] - 9s 1ms/step - loss: 1.3509 - acc: 0.3695 - val_loss: 1.7392 - val_acc: 0.3674
Epoch 6/100
8000/8000 [==============================] - 9s 1ms/step - loss: 1.2755 - acc: 0.3892 - val_loss: 1.7097 - val_acc: 0.3993
Epoch 7/100
8000/8000 [==============================] - 9s 1ms/step - loss: 1.2089 - acc: 0.4097 - val_loss: 1.6694 - val_acc: 0.4104
Epoc

## Usare il modello per tradurre del testo

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
reverse_it_char_index = dict(
    (i, char) for char, i in it_token_index.items())
reverse_en_char_index = dict(
    (i, char) for char, i in en_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, en_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_en_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
def encode(text):
  text_encoded = np.zeros((max_encoder_seq_length, num_encoder_tokens))

  for c, char in enumerate(text):
        text_encoded[c, it_token_index[char]] = 1.
      
  return np.array([text_encoded])

In [0]:
text = "io ho fame"
input_seq = encode(text)
decoded_sentence = decode_sequence(input_seq)

print('Italiano:', text)
print('English:', decoded_sentence)

Italiano: io ho fame
English: i am hungry

